# Axisym PTST with unit ligament area

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34_sw2 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization - Displacement-controlled

In [6]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (np.pi * 2)
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [7]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params =  dict(
     E_T=1, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=1, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [8]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=-.5)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, 0, -1, 0],
                  var='f', dims=[0], value=-1)

In [9]:
loaded_lower_dof = loaded_lower.dofs[0]

In [10]:
loaded_lower_bc = BCDof(var='f', dof=loaded_lower_dof, value = -1 )
unloaded_inner = BCSlice(slice=xd_lower.mesh[0, :, 0, :],
                  var='u', dims=[0], value=0)
bc1 = [axisym, fixed_upper_support, loaded_lower_bc]

In [11]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 1
s.tstep.fe_domain.serialized_subdomains

In [12]:
#bond_m.interact()

In [13]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00u_N, u_T [[0. 0.]] [[0. 0.]] [[0. 0.]]
sig_ [[[-0. -0.]
  [-0. -0.]]]
(0), 
	 1.00u_N, u_T [[0. 0.]] [[0. 0.]] [[0. 0.]]
sig_ [[[-0. -0.]
  [-0. -0.]]]
u_N, u_T [[-1.69008728e-05 -2.24810305e-05]] [[0.99985007 1.00014993]] [[0. 0.]]
sig_ [[[ 9.99850071e-01 -1.69008728e-05]
  [ 1.00014993e+00 -2.24810305e-05]]]
(1), 



# Plot the punch-through curve

In [14]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [15]:
xd12.o_Eia

array([[[ 2,  3],
        [ 6,  7]],

       [[ 8,  9],
        [12, 13]]])

In [16]:
m.hist.U_t[-1][xd12.o_Eia]

array([[[-9.99850071e-01,  3.52533164e-05],
        [-1.00037377e+00,  2.47372491e-05]],

       [[ 0.00000000e+00,  1.83524437e-05],
        [-2.23837662e-04,  2.25621863e-06]]])

In [17]:
xd12.map_U_to_field(U_to[-1])[0,0]

array([ 9.99850071e-01, -1.69008728e-05])

In [18]:
F_to.shape

(2, 16)

In [19]:
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_right_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
U_inner_left_t = np.average(U_to[:, unloaded_inner.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [20]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_right_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_left_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_right_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [22]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [23]:
var_names

dict_keys(['w_pi', 's_pi_x', 's_pi_y', 'z', 'alpha_x', 'alpha_y', 'omega_T', 'omega_N', 'sig_pi', 'tau_pi_x', 'tau_pi_y', 'Z', 'X_x', 'X_y', 'Y_T', 'Y_N'])

In [24]:
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']

In [25]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [26]:
_, (ax, ax_omega) = plt.subplots(1,2, figsize=(10,5))
idx = -1
ax_u = ax.twinx()
ax.plot(x_m, sig_pi_N[idx,:,:].flatten(), color='blue', label='sig')
ax_u.plot(x_m, u_pi_N[idx,:,:].flatten(), color='blue', linestyle='dashed', label='sig')
ax.plot(x_m, sig_pi_Tx[idx,:,:].flatten(), 'o-', color='red', label='tau')
ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)
ax_u.plot(x_m, u_pi_Tx[idx,:,:].flatten(), color='red', linestyle='dashed', label='u_Tx')
ax_omega.plot(x_m, omega_Nx[idx,:,:].flatten(), color='green', label='omega_N')
ax_omega.plot(x_m, omega_Tx[idx,:,:].flatten(), color='green', linestyle='dashed', label='omega_T')
ax.legend()
ax_omega.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-26-df7fb91e129e>:7: MatplotlibDeprecationWarning: Since 3.2, the parameter *where* must have the same size as [0.0 1.0] in fill_betweenx(). This will become an error two minor releases later.
  ax.fill_between(x_m.flatten().flatten(), sig_pi_Tx[idx,:,:].flatten(), 0, 'o-', color='red', alpha=0.2)


In [25]:
sig_pi_Tx[-1]

array([[ 3.69767199,  3.44363753],
       [ 3.44363753,  2.75394758],
       [ 2.75394758,  1.23215998],
       [ 1.23215998, -0.55897674],
       [-0.55897674, -2.55754274],
       [-2.55754274, -3.9001756 ],
       [-3.9001756 , -5.14633512],
       [-5.14633512, -6.6572502 ]])

# FE discretization - Force-controlled